In [2]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
from sklearn.datasets import fetch_openml
import pandas as pd


# Load the Adult dataset
adult = fetch_openml(name='adult', version=2, as_frame=True)
data = adult.data
target = adult.target


In [48]:
# Encode categorical variables
data_encoded = pd.get_dummies(data, drop_first=True)
#print(data_encoded['sex_Female'].value_counts())
print(data_encoded['sex_Male'].value_counts())
target_encoded = target.map({'>50K': 1, '<=50K': 0})

print(target_encoded)
full_df = data_encoded.copy()
full_df['income'] = target_encoded
empty_rows_count = full_df.isna().sum()
print(empty_rows_count)
full_df.dropna(inplace=True)
print(full_df)

sex_Male
True     32650
False    16192
Name: count, dtype: int64
0        0
1        0
2        1
3        1
4        0
        ..
48837    0
48838    1
48839    0
48840    0
48841    1
Name: class, Length: 48842, dtype: category
Categories (2, int64): [0, 1]
age                               0
fnlwgt                            0
education-num                     0
capital-gain                      0
capital-loss                      0
                                 ..
native-country_Trinadad&Tobago    0
native-country_United-States      0
native-country_Vietnam            0
native-country_Yugoslavia         0
income                            0
Length: 98, dtype: int64
        age    fnlwgt  education-num  capital-gain  capital-loss  \
0      25.0  226802.0            7.0           0.0           0.0   
1      38.0   89814.0            9.0           0.0           0.0   
2      28.0  336951.0           12.0           0.0           0.0   
3      44.0  160323.0           10.0        768

In [49]:
from aif360.datasets import BinaryLabelDataset

# Create a BinaryLabelDataset with the corrected DataFrame
dataset = BinaryLabelDataset(df=full_df,
                             label_names=['income'],
                             protected_attribute_names=['sex_Male'], 
                             favorable_label=1,
                             unfavorable_label=0)


In [50]:
from aif360.metrics import BinaryLabelDatasetMetric

privileged_groups = [{'sex_Male': 1}]
unprivileged_groups = [{'sex_Male': 0}]



metric = BinaryLabelDatasetMetric(dataset, 
                                  unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups)

print("Statistical parity difference:", metric.statistical_parity_difference())
print("Disparate impact:", metric.disparate_impact())


Statistical parity difference: -0.19451574596420296
Disparate impact: 0.3596552625800337
